### Content Based system

Content Based Recommender system is using the watching or listening history of the user, through the various features's score of items and the user's rating to different items, to get the preference of the user to different styles of movies, musics or other things. So each customer should have their own model according to their own history. When system wants to reommend some new things, it can input new items' feature value or score into the model, and finding the the most similar item in the user's viewing history to predict the rating of the new item, and recommends the new things with higher predicted rating.

In [73]:
import numpy as np
import pandas as pd

In [74]:
user_rate = pd.read_csv('/Users/yumengxiao/Documents/7374/Assignment3/hetrec2011-lastfm-2k/user_artists.csv')
user_rate.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


The 'weight' is actually the listening count for each (user, artist) pair, so if weight is higher it will mean that the user likes this artist more. But beacause the range of weight is too big, we deicide to tranform it into integer 1 to 4 by using the quartile.

In [75]:
user_rate['weight'].describe()

count     92834.00000
mean        745.24393
std        3751.32208
min           1.00000
25%         107.00000
50%         260.00000
75%         614.00000
max      352698.00000
Name: weight, dtype: float64

In [76]:
user_rate['weight'] = user_rate['weight'].apply(lambda x: 1 if x <= 107 
                                    else (2 if x <= 260 
                                    else (3 if x <= 614 
                                    else  4)))
user_rate.head()

,userID,artistID,weight
0,2,51,4
1,2,52,4
2,2,53,4
3,2,54,4
4,2,55,4


In [77]:
# Using the history of the user whose ID is 3
user_rate = user_rate[user_rate['userID']== 3]
user_rate.head()

,userID,artistID,weight
50,3,101,4
51,3,102,4
52,3,103,3
53,3,104,3
54,3,105,3


In [78]:
#Get all the tags of each artist, each artist has more than one tag
artist_tag = pd.read_csv('/Users/yumengxiao/Documents/7374/Assignment3/hetrec2011-lastfm-2k/user_taggedartists.csv')
artist_tag = artist_tag[['artistID','tagID']]

tag = pd.read_csv('/Users/yumengxiao/Documents/7374/Assignment3/hetrec2011-lastfm-2k/tags.csv',encoding= 'ISO-8859-1')
artist_tag = pd.merge(artist_tag, tag,on = 'tagID', how = 'inner', sort = False)

artist_tag.head()

,artistID,tagID,tagValue
0,52,13,chillout
1,63,13,chillout
2,73,13,chillout
3,94,13,chillout
4,6177,13,chillout


In [79]:
# merge each rating records of user 3 with all the received tags from the system of the artist.
user_artist_tag = pd.merge(user_rate, artist_tag, on = 'artistID', how = 'inner')
user_artist_tag.head()

,userID,artistID,weight,tagID,tagValue
0,3,101,4,15,downtempo
1,3,101,4,14,ambient
2,3,101,4,33,experimental
3,3,101,4,44,glitch
4,3,101,4,45,u-cover


In [80]:
tagValue = user_artist_tag['tagValue'].drop_duplicates()
len(tagValue)

89

In [81]:
artist = user_artist_tag['artistID'].drop_duplicates()
len(artist)

26

So user 3 has listened 26 artists' music and these 26 artists have received 89 kinds of tags.

Because the format of the dataset now is one tag a row, we have to tranform it into one artist one row with numerious columns of tags. Each value inside the table is the number of times the artist was given the tag.

In [82]:
df_artist_tag = pd.DataFrame(np.zeros((26,89)), index = artist, columns = tagValue )
df_artist_tag.head()

tagValue,downtempo,ambient,experimental,glitch,u-cover,melancholy,idm,vu-us,female vocalist,modern classical,...,neo-classical underground,cool,icelandic,canada,mistagged track,lounge,nu jazz,special,vfsix,post rock
artistID,,,,,,,,,,,,,,,,,,,,,
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


So we use the (index, column) for each value in the table above to get each pairs of (artistID, tag), and find out the number of this pair in the table user_artist_tage.

In [83]:
columns = 89
rows = 26

for i in range(0,rows):

    for j in range(0,columns):

        f = user_artist_tag[(user_artist_tag['artistID']== df_artist_tag.index[i]) & (user_artist_tag['tagValue'] == df_artist_tag.columns[j])]
        if  f.shape[0] != 0:
            df_artist_tag.iloc[i,j] = f.shape[0]

df_artist_tag.head()

tagValue,downtempo,ambient,experimental,glitch,u-cover,melancholy,idm,vu-us,female vocalist,modern classical,...,neo-classical underground,cool,icelandic,canada,mistagged track,lounge,nu jazz,special,vfsix,post rock
artistID,,,,,,,,,,,,,,,,,,,,,
101,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


For example, the 6 with airtistID of 103 and tag of ambient means the pair of (103, ambient) has appeared 6 times in the user_artist_tage, which also means that airtist 103 has been assigned the tag 'ambient' 6 times .

In [100]:
# merge the new table with rating records
df = pd.merge(user_rate, df_artist_tag, left_on = 'artistID', right_index = True, how = 'inner').reset_index(drop=True)
df.head()

,userID,artistID,weight,downtempo,ambient,experimental,glitch,u-cover,melancholy,idm,...,neo-classical underground,cool,icelandic,canada,mistagged track,lounge,nu jazz,special,vfsix,post rock
0,3,101,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,102,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,103,3,0.0,6.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,104,3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,106,3,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Then we choose to use KNN to build a prediction model. KNN is finding k points or records that most nearest to the test data, and getting all the labels of these k records and using the label with highest frequency as the final category of the test data.

In [101]:
from sklearn.model_selection import train_test_split
from sklearn import neighbors

df_x = df.iloc[:,3:]
df_y = df.iloc[:,2]

# meger the artistID with features for outputing the ranking of artist at last
df_x_artist = pd.merge(df['artistID'],df_x, left_index = True, right_index = True, how = 'inner')
df_x_artist.head()

,artistID,downtempo,ambient,experimental,glitch,u-cover,melancholy,idm,vu-us,female vocalist,...,neo-classical underground,cool,icelandic,canada,mistagged track,lounge,nu jazz,special,vfsix,post rock
0,101,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,102,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,103,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,104,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,106,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
X_train_artist, X_test_artist, y_train, y_test = train_test_split(df_x_artist, df_y ,test_size = 0.5)

# drop the artistID column and then input the data into model
X_train = X_train_artist.drop(['artistID'], axis = 1, inplace = False)
X_test =  X_test_artist.drop(['artistID'], axis = 1, inplace = False)

In [117]:
# finding 5 records that most nearest to the test data,
clf = neighbors.KNeighborsClassifier(5)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.7692307692307693

The accuracy of this multi-class classification model is about 77%. If we can more history record of each user, the accuracy must can still be increased.

In [127]:
y_pred = clf.predict(X_test)
y_pred = pd.Series(y_pred.tolist())

In [119]:
test_artistID  =  X_train_artist['artistID'].reset_index(drop=True)

In [133]:
# merge the artistID and predidcted result of tets data and sort the rating
result = pd.concat([test_artistID, y_pred], axis = 1, ignore_index = False)
result.columns = ['artistID', 'predicted_rating']
result = result.sort_values(by="predicted_rating", ascending = False)

#output the top 3 artist
result['artistID'].head(3)

2     132
10    142
12    128
Name: artistID, dtype: int64

So we would recommend artist 132, 142 128 to user 3